In [107]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

from tqdm import tqdm_notebook
from IPython import display

import warnings
warnings.filterwarnings("ignore")

In [119]:
path = r'C:\Users\Public\Documents\HRS\Features new'
beginning_of_name_EL = '\Feature matrix EL no noise'
beginning_of_name_EL_OM = '\Feature matrix EL+OM no noise'
end_of_name = 'peaks.csv'
list_of_names = [' 3 ', ' 4 ', ' 5 ']

f1 = np.zeros((3,2)) # list of f1_scores, shape list_of_names*3

for i in range(len(list_of_names)):
    data = pd.read_csv(path + beginning_of_name_EL + list_of_names[i] + end_of_name, index_col=0)
    data.drop(['Label for OM and EL'], axis=1, inplace=True)
    data1 = pd.read_csv(path + beginning_of_name_EL_OM + list_of_names[i] + end_of_name, index_col=0)
    print(list_of_names[i] + end_of_name)
    
    X = data.drop('target', axis=1) #full of EL data ~ 900 sequencies
    y = data['target']

    X1 = data1.drop('target', axis=1) #full of EL+OM data ~ 200 sequencies
    y1 = data1['target']

    scaler = StandardScaler()
    
    X = scaler.fit_transform(X)
    X1 = scaler.fit_transform(X1)
     

    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.3, stratify=y)

    (X1_train, X1_test, y1_train, y1_test) = train_test_split(X1, y1,\
                                                                  test_size=0.3, stratify=y1)
    
    X_resampled, y_resampled = RandomUnderSampler().fit_resample(X_train, y_train)
    print(X_resampled.shape, y_resampled.shape)
    X1_resampled, y1_resampled = RandomUnderSampler().fit_resample(X1_train, y1_train)
    print(X1_resampled.shape, y1_resampled.shape)

    
#     parameters = {'kernel':('linear', 'rbf'), 'C':[1, 2, 5, 7, 10]}


    clf = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced') #,
    
    svc_f1_score = cross_val_score(clf, X_resampled, y_resampled, scoring='f1', cv=10)
    svc_f1_score1 = cross_val_score(clf, X1_resampled, y1_resampled, scoring='f1', cv=10)

    svc_mean = svc_f1_score.mean()
    svc_mean1 = svc_f1_score1.mean()
    
    print("SVC model EL f1: mean %.3f, std %.3f" % (svc_f1_score.mean(), svc_f1_score.std(ddof=1)))
    print("SVC model EL+OM f1: mean %.3f, std %.3f" % (svc_f1_score1.mean(), svc_f1_score1.std(ddof=1)))
    
    svc_mean_std = svc_f1_score.std(ddof=1)/np.sqrt(len(svc_f1_score))
    svc_mean_std1 = svc_f1_score1.std(ddof=1)/np.sqrt(len(svc_f1_score1))

    print("SVC model EL mean f1 95%% confidence interval:", _tconfint_generic(svc_mean, svc_mean_std,
                                                                       len(svc_f1_score) - 1,
                                                                       0.05, 'two-sided'))

    print("SVC model EL+OM mean f1 95%% confidence interval:", _tconfint_generic(svc_mean1, svc_mean_std1,
                                                                       len(svc_f1_score1) - 1,
                                                                       0.05, 'two-sided'))
    
#     optimizer = GridSearchCV(clf, parameters, cv=10, scoring='f1')
#     optimizer.fit(X_train, y_train) 
#     print(optimizer.best_score_, optimizer.best_params_)
    
    

#     optimizer1 = GridSearchCV(clf, parameters, cv=10, scoring='f1')
#     optimizer1.fit(X1_train, y1_train) 
#     print(optimizer1.best_score_, optimizer1.best_params_)


#     y_pred = optimizer.best_estimator_.predict(X_test)
#     y_pred1 = optimizer1.best_estimator_.predict(X1_test)

#     f1[i][0]= f1_score(y_test, y_pred)
#     f1[i][1]= f1_score(y1_test, y_pred1)

    

 3 peaks.csv
(78, 15) (78,)
(68, 30) (68,)
SVC model EL f1: mean 0.570, std 0.149
SVC model EL+OM f1: mean 0.603, std 0.276
SVC model EL mean f1 95%% confidence interval: (0.463633771639035, 0.6761064881012246)
SVC model EL+OM mean f1 95%% confidence interval: (0.4059580531161614, 0.8010260738679656)
 4 peaks.csv
(78, 19) (78,)
(68, 38) (68,)
SVC model EL f1: mean 0.498, std 0.170
SVC model EL+OM f1: mean 0.591, std 0.195
SVC model EL mean f1 95%% confidence interval: (0.3756861414673876, 0.6193499335686874)
SVC model EL+OM mean f1 95%% confidence interval: (0.4515235675099485, 0.7311171251307442)
 5 peaks.csv
(78, 23) (78,)
(68, 46) (68,)
SVC model EL f1: mean 0.477, std 0.208
SVC model EL+OM f1: mean 0.536, std 0.147
SVC model EL mean f1 95%% confidence interval: (0.32752839080790663, 0.6256462123666965)
SVC model EL+OM mean f1 95%% confidence interval: (0.4311376766945314, 0.6415607360038813)


In [95]:
optimizer.cv_results_['mean_test_score'].mean()

0.49894993894993894

In [118]:
f1

array([[0.        , 0.28571429],
       [0.09302326, 0.21428571],
       [0.13953488, 0.14285714]])

In [116]:
emoji_matrix = []
for i in range(f1.shape[0]):
    if f1[i][0] > f1[i][1]:
        emoji_matrix.append('beeee')
    else:
        emoji_matrix.append('yraaaa')
emoji_matrix

['yraaaa', 'yraaaa', 'yraaaa']